![logo](./img/LogoLine_horizon_C3S.png)

<br>

# Download Reanalysys: ERA5

### About


The notebook has the following outline:
* 1 - Download data from the CDS
* 2 - Hindcast data processing: calculate the reference climate mean
* 3 - Real-time forecasts: calculate seasonal forecast anomalies
* 4 - Visualize seasonal forecast monthly anomalies for a geographical subregion
  * 4.1 - Spatial maps
  * 4.2 - Time series of regional averages
* 5 - Reproduce C3S graphical products: compute 3-month anomalies

<br>

### Install packages

In [ ]:
# Install CDS API for downloading data from the CDS
#!pip install cdsapi

In [ ]:
# Install cfgrib to enable us to read GRIB format files
#!conda install -c conda-forge cfgrib -y

In [ ]:
# Miscellaneous operating system interfaces
import os

# CDS API
import cdsapi

# To map GRIB files to NetCDF Common Data Model
import cfgrib

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Interactive HTML widgets
import ipywidgets as widgets

# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()

<hr>

In [ ]:
## 1. Request data from the CDS programmatically with the CDS API

The first step is to request data from the Climate Data Store programmatically with the help of the CDS API. Let us make use of the option to manually set the CDS API credentials. First, you have to define two variables: `URL` and `KEY` which build together your CDS API key. Below, you have to replace the `#########` with your personal CDS key. Please find [here](https://cds.climate.copernicus.eu/api-how-to) your personal CDS key.

In [ ]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '5596:e5a0ace8-18ac-417c-96a5-ecb447a7eb20'
#year = 1993

Here we specify a data directory in which we will download our data and all output files that we will generate:

In [ ]:
DATADIR = '/home/daniel/Documents/intoDBP/input/reanalysis'
print(DATADIR)

In [ ]:
c = cdsapi.Client(url=URL, key=KEY)

In [ ]:
var_short = 'tp'
for year in np.arange(1993,2024):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': 'total_precipitation',
        'year': str(year),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_era5_' + str(year) +' _hourly_' + var_short + '.grib')
    

2024-04-24 14:17:00,309 INFO Welcome to the CDS
2024-04-24 14:17:00,311 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-24 14:17:00,426 INFO Request is completed
2024-04-24 14:17:00,428 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data4/adaptor.mars.internal-1713959875.4214723-7725-16-b9740c73-40b1-42d9-81b1-b2f22fb0026b.grib to /home/daniel/Documents/intoDBP/input/reanalysis/ecmwf_era5_1993 _hourly_tp.grib (2M)
2024-04-24 14:17:00,820 INFO Download rate 5.1M/s                                                                                                                                                                  
2024-04-24 14:17:00,918 INFO Welcome to the CDS
2024-04-24 14:17:00,918 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-24 14:17:00,974 INFO Request is queued


In [ ]:
var_short = 't2m'
for year in np.arange(1993,2024):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': '2m_temperature',
        'year': str(year),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_era5_' + str(year) +' _hourly_' + var_short + '.grib')
    